In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline

train_data = pd.read_csv('Dataset/train.csv', index_col='pet_id')
test_data = pd.read_csv('Dataset/test.csv', index_col='pet_id')



from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

train_data['color_type_encoded'] = encoder.fit_transform(train_data.color_type)
test_data['color_type_encoded'] = encoder.transform(test_data.color_type)


np.random.seed(42)

train_condition=np.random.choice(np.arange(0, 3), p=[6281/17357,6819/17357,4257/17357], size=1477)



test_condition = np.random.choice(np.arange(0,3), p=[2685/7453,2928/7453,1840/7453], size=619)

train_data['condition_filled'] = 0
test_data['condition_filled'] = 0

train_data.loc[train_data.condition.isnull(), 'condition_filled'] = 1
test_data.loc[test_data.condition.isnull(), 'condition_filled'] = 1

train_data.loc[train_data.condition.isnull(), 'condition'] = train_condition
test_data.loc[test_data.condition.isnull(), 'condition'] = test_condition

train_data['issue_date'] = pd.to_datetime(train_data.issue_date)
train_data['listing_date'] = pd.to_datetime(train_data.listing_date)
                                            
test_data['issue_date'] = pd.to_datetime(test_data.issue_date)
test_data['listing_date'] = pd.to_datetime(test_data.listing_date)

train_data['issue_year'] = train_data.issue_date.dt.year
train_data['issue_month'] = train_data.issue_date.dt.month
train_data['issue_day'] = train_data.issue_date.dt.day

train_data['listing_year'] = train_data.listing_date.dt.year
train_data['listing_month'] = train_data.listing_date.dt.month
train_data['listing_day'] = train_data.listing_date.dt.day

test_data['issue_year'] = test_data.issue_date.dt.year
test_data['issue_month'] = test_data.issue_date.dt.month
test_data['issue_day'] = test_data.issue_date.dt.day

test_data['listing_year'] = test_data.listing_date.dt.year
test_data['listing_month'] = test_data.listing_date.dt.month
test_data['listing_day'] = test_data.listing_date.dt.day

train_data['total_days'] = np.array(train_data.listing_date-train_data.issue_date).astype('timedelta64[D]').astype('int64')
test_data['total_days'] = np.array(test_data.listing_date-test_data.issue_date).astype('timedelta64[D]').astype('int64')

train_data['length(cm)'] = train_data['length(m)']*100
test_data['length(cm)'] = test_data['length(m)']*100

train_data['rec_area'] = train_data['length(cm)'] * train_data['height(cm)']
test_data['rec_area'] = test_data['length(cm)'] * test_data['height(cm)']

In [2]:
features = ['condition', 'length(cm)', 'height(cm)','rec_area', 'X1', 'X2', 'color_type_encoded',
           'issue_year', 'issue_month', 'issue_day', 'listing_year', 'listing_month', 'listing_day', 'total_days']

In [3]:
from sklearn.preprocessing import normalize
X = normalize(train_data[features])

In [59]:
X[0]

array([6.84829819e-04, 2.73931928e-02, 2.66398800e-03, 2.13119040e-01,
       4.45139383e-03, 3.08173419e-03, 6.16346837e-03, 6.90308458e-01,
       2.39690437e-03, 3.42414910e-03, 6.90308458e-01, 3.08173419e-03,
       7.19071310e-03, 2.49962884e-02])

In [4]:
y_pet = np.array(train_data['pet_category'])-1
y_breed = np.array(train_data['breed_category'])

In [60]:
train_data.head()

,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category,...,condition_filled,issue_year,issue_month,issue_day,listing_year,listing_month,listing_day,total_days,length(cm),rec_area
pet_id,,,,,,,,,,,,,,,,,,,,,
ANSL_69903,2016-07-10,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1,...,0,2016,7,10,2016,9,21,73,80.0,622.40
ANSL_66892,2013-11-21,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2,...,0,2013,11,21,2018,12,27,1862,72.0,1021.68
ANSL_69750,2014-09-28,2016-10-19 08:24:00,1.0,Brown,0.15,40.90,15,4,2.0,4,...,1,2014,9,28,2016,10,19,752,15.0,613.50
ANSL_71623,2016-12-31,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2,...,0,2016,12,31,2019,1,25,755,62.0,1104.84
ANSL_57969,2017-09-28,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1,...,0,2017,9,28,2017,11,19,52,50.0,553.00


In [7]:
dummy_y = np_utils.to_categorical(y_pet)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [36]:
def baseline_model(output_channels):
    model = Sequential()
    model.add(Dense(16, input_dim=14,activation='relu'))
    model.add(Dense(32, input_dim=14,activation='relu'))
    model.add(Dense(16, input_dim=14,activation='relu'))
    model.add(Dense(output_channels, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    
    return model

In [37]:
estimator = KerasClassifier(build_fn=baseline_model(4), epochs=10, batch_size=20, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator,X, dummy_y, cv=kfold,)
print("Baseline:%.2f%%(%.2f%%)"%(results.mean()*100, results.std()*100))

Baseline:nan%(nan%)


In [14]:
model = Sequential()
model.add(Dense(128, input_dim=14,activation='relu'))
model.add(Dense(64, input_dim=14,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, input_dim=14,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
EPOCH = 10

In [ ]:
history = model.fit(X, dummy_y, batch_size=8, epochs=EPOCH, verbose=0)

In [ ]:
history.history

In [31]:
dummy_y

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [87]:
y_pet[100:120]

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 3, 0, 0, 3, 1, 0, 1, 0, 1, 0, 1],
      dtype=int64)

In [88]:
history

In [91]:
history.history

{'loss': [0.6663735049715515], 'accuracy': [0.71482426]}